# To Do List - Finish by Sunday. 

## 1. Finish writing the constraints, variables, and parameters with domains. 
## 2. Continue trying to figure out how to work with indexed tuple sets
## 3. Begin entering data into a .dat file
## 4. Figure out if we can linearize a quadratic & linearization for logic constraints. 

In [25]:
# Import Pyomo Modules

from __future__ import division
from pyomo.environ import *
import random
import numpy as np



In [26]:
# Define an empty AbstractModel() object
model = AbstractModel()

In [27]:
# Define the index sets for the grid, time horizon, and age class.

model.Iset = RangeSet(3) #e.g i = {1, 2, 3}
model.Jset = RangeSet(3)
model.Tset = RangeSet(7)
model.Kset = RangeSet(50)

0


In [28]:
# Defining all model parameters. 
# ALL PARAMETERS WILL MOST LIKELY NEED TO BE INDEXED AND SPECIFY A DEFAULT VALUE?

model.juvsurv = Param(within=NonNegativeIntegers)   # Juvenille Tree Survivorship Probability
model.juvdeath = Param()  # Juvenille Tree Mortality Probability (model.juvdeath = 1 - model.juvsurv)
model.inventory = Param() # Tree inventory that is constant for each cell - acts as carrying capacity
model.beta = Param()      # Failure Rate of Infestation
model.R = Param()         # Growth rate of MPB per effective tree --> 1.8 per year, not indexed by time yet. 
model.gamma = Param()     # Inter-Resource Competition Parameter
model.lamb = Param()      # Effectiveness of Level 1 Control
model.rho = Param()       # Effectivneness of Level 2 Control
model.disper = Param()    # Proportion of dispersal 
model.alpha = Param()     # Response Function Alpha 


model.c0 = Param()        # Fitting parameter for dispersal
model.c1 = Param()        # Fitting parameter for dispersal

# Tester Cell

In [29]:
model.A = RangeSet(7)

# Code Mapping to Model Variables

### Note: Maybe put this in a separate file - not showing up how I would like it to in Markdown

### $$ model.juvenille -->  j_{k,t}^{(i,j)}$$ 

### $$ model.juvenilleTotal -->  J_{t}^{(i,j)} = \sum_{k=1}^{NJ}j_{k,t}^{(i,j)}, k = 1,2,...,NJ-1 $$

### $$ model.redsnag --> I_{t-1} $$

### $$ model.greysnag --> I_{t-2} $$

# Coding All Model Variables:

In [ ]:
model.juvenille = Var(model.Iset, model.Jset, model.Tset, model.Kset,          
                      within=NonNegativeReals, initialize='''FILL IN LATER''')

def juv_total(model, i, j, t, k):
    return sum(model.juvenille[k] for k in range(1,50))
model.juvenilleTotal = Var(model.Iset, model.Jset, model.Tset, model.Kset,
                           within=NonNegativeReals, initialize=juv_total)

model.susceptible = Var(model.Iset, model.Jset, model.Tset,
                       within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.inf_b4travel = Var(model.Iset, model.Jset, model.Tset,
                    within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.inf_posttravel = Var(model.Iset, model.Jset, model.Tset,
                    within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.inf_treated = Var(model.Iset, model.Jset, model.Tset,
                    within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.redsnag = Var(model.Iset, model.Jset, model.Tset, model.Kset,
                   within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.greysnag = Var(model.Iset, model.Jset, model.Tset, model.Kset,
                    within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.treeinventory = Var(model.Iset, model.Jset, model.Tset,
                        within=NonNegativeReals, initialize = '''FILL IN LATER w/ expr?''')

model.ownwweight = Var(model.Iset, model.Jset, model.Tset,
                     within=NonNegativeReals, initialize = '''FILL IN LATER w/ Expression''')

model.outweight = Var(model.Iset, model.Jset, model.Tset,
                     within=NonNegativeReals, initialzie = ''' FILL IN LATER w/ Expression ''')

model.levelone =  Var(model.Iset, model.Jset, model.Tset,
                     within=NonNegativeReals, initialzie = ''' FILL IN LATER w/ Expression ''')

model.leveltwo =  Var(model.Iset, model.Jset, model.Tset,
                     within=NonNegativeReals, initialzie = ''' FILL IN LATER w/ Expression ''')

# Coding Constraints

## Juvenille Constraints

In [ ]:
# Constraint 1: juvenilles that advance to the next age class (eq. 1)
def juv_advance(model, i, j, t, k):
    return model.juvenille[i,j,t+1,k+1] == model.juvenille[i,j,t,k]*model.juvsurv

# Constraint 2: total number of juvenilles in all age classes on cell (i,j)
def juv_total_sum(model, i, j, t, k):
    return """ This constraint is actually a variable declaration """

# Constraint 3: recruitment of seedlings to the first juvenille age class. 
def juv_recruit(model, i, j, t, k): """ Not sure how you would specify the I_t-2 var, will come back to this """
    return model.juvenille[i,j,t,1] == model.juvdeath*model.juvenilleTotal + model.greysnag

## Susceptible Constraints


In [ ]:
# Constraint 5: Susceptible recruitment 
def susceptible_advance(model, i, j, t, k):
    return model.susceptible[i,j,t+1] == model.susceptible[i,j,t] - model.infected[i,j,t] \\
            + model.juvsurv*model.juvenille[i,j,t,k==50]

## Dispersal Constraints

In [ ]:
# Constraint 7: Own cell dispersal weight. 
def own_cell_dispersal(model, i, j, t):
    return model.ownweight == (1-8*model.disper)*model.inf_beforetreat[i,j,t]

# Constraint 8:
def out_cell_dispersal(model):
    return model.outweight """ Will need to finish later """

## Infestation Constraints

In [ ]:
# Constraint 10: Post dispersal population in cell (i,j)
def postmigration_infection(model, i, j, t):
    return model.inf_posttravel == model.ownweight[i,j,t] + model.outweight[i,j,t]

# Constraint 11: infected population after cell treatment. 
def posttreatment_infection(model, i, j, t):
    return model.inf_treated == (1 - model.lamb*model.levelone - model.rho*model.leveltwo)*model.inf_posttravel

# Constraint 4: Infestation growth:
def infest_growth(model, i, j, t): 
    return model.infb4travel[i, j, t+1] == model.susceptible[i,j,t] \\
                                        * (1 / ( (model.inf_treated ** model.inf_treated) * model.alpha) ) \\
                                        * (model.inf_treated ** model.inf_treated)

In [ ]:
# Constraint 6: Tree inventory - constant for each cell. 
def tree_inventory(model, i, j, t):
    return model.inventory = model.juvenilleTotal[i,j,t+1] + model.susceptible[i,j,t+1] + model.infected[i,j,t] \\
                            + model.greysnag[i,j,t]

# Constraint 9:
def adding_up(model):
    return




In [ ]:
# A functino to try and generate random shocks - will come back to this after I finish coding the model without it. 

################################################################################################################
# Random Parameter Generation for Long Range Dispersal 
# !!! NEED TO ASK IN STACK OVERFLOW !!!
#def Randlrd_init(model,i,j):
#    mu = []
#    for i in range(1,3):
#        for j in range(1,3):
#            new_rand = random.randrange(0, 0.1, 1)
#            mu.append(new_rand)
#    return mu
    
#model.v = Param(model.Iset, model.Js, model.Ts,     # Random Long Range Dispersal Parameter
 #                   initialize=Randlrd_init, default=0)     
################################################################################################################

# Master List of Constraints